In [0]:
#!/usr/bin/python


import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import demultiplier as dem
import encoder as enc
import generator as gen
import discriminator as disc
import STL10GrayColor as STLGray
import utils as utls
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import lab2rgb

In [2]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#data
transform = transforms.Compose([transforms.Resize(128)])#,
#                                transforms.ToTensor()])

# Load STL10 dataset
stl10_trainset = STLGray.STL10GrayColor(root="./data",
                              split='train',
                              download=True,
                              transform=transform)

Files already downloaded and verified


In [0]:
# Parameters
batch_size = 32
z_dim = 512
params_loader = {'batch_size': batch_size,
               'shuffle': False}

train_loader = DataLoader(stl10_trainset, **params_loader)


In [0]:
demultiplier = dem.Demultiplier()
demultiplier = demultiplier.to(device)

encoder = enc.Encoder()
encoder = encoder.to(device)

generator = gen.Generator()
generator.apply(utls.weights_init)
generator = generator.to(device)

discriminator = disc.Discriminator()
discriminator.apply(utls.weights_init)
discriminator = discriminator.to(device)

criterion = nn.MSELoss()

optimizer_m = torch.optim.Adam(demultiplier.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_e = torch.optim.Adam(encoder.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_g = torch.optim.Adam(generator.parameters(), lr=0.0001, betas=(0.5, 0.999))
optimizer_d = torch.optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))


In [0]:
real_label, fake_label = (1, 0)
i = 0
n_epochs = 15
D_losses = []
G_losses = []
plot_this_shit = True

for i in range(0,n_epochs) :
  print("epoch ", i)
  i+=1
  
  j=0
  
  for batch_index, (image_g, image_c) in enumerate(train_loader):

      image_g = image_g.to(device)
      image_c = image_c.to(device)

      # train discriminator with real
      #print("training discriminator with real")

      discriminator.zero_grad()

      d_real_output = discriminator(image_c)

      labels = torch.full((image_c.shape[0],), real_label, device=device) #device = device
      errD_real = criterion(d_real_output, labels)
      errD_real.backward()

      # demultiply

      image_g_dem = demultiplier(image_g)

      #print('Demultiplied')
      #print(image_g.shape)
      # generate color image

      enc_output = encoder(image_g_dem)

      #print('encoded')
      #print(enc_output.shape)

      color_output = generator(enc_output)

      #print('decoded')
      #print(color_output.shape)

      # train discriminator with fake
      #print("training discriminator with fake")

      #why detach ? https://github.com/pytorch/examples/issues/116
      d_fake_output = discriminator(color_output.detach())

      #print("discriminated")
      #print(d_fake_output.shape)

      labels = torch.full((image_c.shape[0],), fake_label, device=device) #device = device

      errD_fake = criterion(d_fake_output, labels)
      errD_fake.backward()
      
      total_errD = errD_fake + errD_real

      optimizer_d.step()

      # train generator
      #print("training generator")

      generator.zero_grad()

      labels.fill_(real_label)

      #this is needed because of the previous detach, line 99.
      d_fake_output = discriminator(color_output)

      errG = criterion(d_fake_output, labels)
      errG.backward()

      optimizer_g.step()
      # train demultiplier

      demultiplier.zero_grad()
      # no idea how to do this

      if (j%10 == 0) :
        print("iteration ", j, "out of ", len(train_loader.dataset)//batch_size, 
              "\terrD : ", round(total_errD.item(),3), "\terrG : ", round(errG.item(),3))
      
      D_losses.append(total_errD.item())
      G_losses.append(errG.item())
      
      if (j == 0 and plot_this_shit):
        with torch.no_grad():
          color_output = generator(enc_output).detach().cpu()
          
          r, c = 5, 5
          index = 0
          fig, axs = plt.subplots(r, c, figsize=(10,10))
          
          img_g = image_g.cpu()
          img_g = img_g.detach()
          img_c = color_output.detach()
          final = utls.convert_lab2rgb(img_g, img_c).cpu()
           
          for a in range(r):
            for b in range(c):

                axs[a,b].imshow(final[index, ...])
                axs[a,b].axis('off')
                
                if (index == 1) :
                  final = final.numpy()
                  print("R min/max : ", np.min(final[index, ..., 0]), np.max(final[index, ..., 0]),
                        "G min/max : ", np.min(final[index, ..., 1]), np.max(final[index, ..., 1]),
                        "B min/max : ", np.min(final[index, ..., 2]), np.max(final[index, ..., 2]))
                
                index += 1
          
          fig.savefig("___epoch_%d.png" % i)
          plt.close()
          print("plotted")
      
      
      j += 1
          


epoch  0
iteration  0 out of  156 	errD :  95.766 	errG :  226.613
R min/max :  0.029262142 0.99119735 G min/max :  0.031419612 0.98831356 B min/max :  0.032994628 0.9898991
plotted
iteration  10 out of  156 	errD :  5.21 	errG :  15.195
iteration  20 out of  156 	errD :  10.218 	errG :  47.144
iteration  30 out of  156 	errD :  42.088 	errG :  43.142
iteration  40 out of  156 	errD :  1.38 	errG :  2.983
iteration  50 out of  156 	errD :  3.285 	errG :  9.643
iteration  60 out of  156 	errD :  4.485 	errG :  1.994
iteration  70 out of  156 	errD :  2.651 	errG :  1.286
iteration  80 out of  156 	errD :  1.288 	errG :  0.896
iteration  90 out of  156 	errD :  5.626 	errG :  2.804
iteration  100 out of  156 	errD :  1.534 	errG :  5.561
iteration  110 out of  156 	errD :  27.147 	errG :  1.991
iteration  120 out of  156 	errD :  1.332 	errG :  1.559
